In [65]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [36]:
patients_df = pd.read_csv("../physionet.org/files/mimiciii/1.4/PATIENTS.csv").drop(columns=['ROW_ID', 'DOD', "DOD_HOSP", 'DOD_SSN'])
print(len(patients_df))
patients_df.head()

46520


,SUBJECT_ID,GENDER,DOB,EXPIRE_FLAG
0,249,F,2075-03-13 00:00:00,0
1,250,F,2164-12-27 00:00:00,1
2,251,M,2090-03-15 00:00:00,0
3,252,M,2078-03-06 00:00:00,0
4,253,F,2089-11-26 00:00:00,0


In [39]:
icustays_df = pd.read_csv("../physionet.org/files/mimiciii/1.4/ICUSTAYS.csv").drop(columns=['ROW_ID','DBSOURCE','FIRST_CAREUNIT','LAST_CAREUNIT','FIRST_WARDID','LAST_WARDID','OUTTIME'])
print(len(icustays_df), len(set(list(icustays_df['SUBJECT_ID']))))
icustays_df.head()

61532 46476


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,INTIME,LOS
0,268,110404,280836,2198-02-14 23:27:38,3.2490
1,269,106296,206613,2170-11-05 11:05:29,3.2788
2,270,188028,220345,2128-06-24 15:05:20,2.8939
3,271,173727,249196,2120-08-07 23:12:42,2.0600
4,272,164716,210407,2186-12-25 21:08:04,1.6202


In [25]:
annotations_df = pd.read_csv("../annotations.csv")
annotations_df.head()

,Hospital.Admission.ID,subject.id,chart.time,cohort,Obesity,Non.Adherence,Developmental.Delay.Retardation,Advanced.Heart.Disease,Advanced.Lung.Disease,Schizophrenia.and.other.Psychiatric.Disorders,Alcohol.Abuse,Other.Substance.Abuse,Chronic.Pain.Fibromyalgia,Chronic.Neurological.Dystrophies,Advanced.Cancer,Depression,Dementia,Unsure
0,118003,3644,118003,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0
1,177830,97736,999999,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,185673,27694,999999,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,131938,16275,131938,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,198999,4059,198999,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0


## List of hospital admissions IDs from annotation

In [26]:
hadmids = list(annotations_df["Hospital.Admission.ID"])
print(len(hadmids), len(set(hadmids)))

1610 1561


In [24]:
counter = 0 
for i in range(len(annotations_df)):
    for j in range(len(annotations_df)):
        if(i!=j and annotations_df['Hospital.Admission.ID'][i] == annotations_df['Hospital.Admission.ID'][j]):
            counter +=1
#             for columns in annotations_df.columns:
#                 print(columns, annotations_df[columns][i], annotations_df[columns][j])
#             print()
print(counter//2)

54


In [27]:
annotations_df_drop = annotations_df.drop_duplicates(subset=['Hospital.Admission.ID']).reset_index(drop=True)
print(len(annotations_df_drop))
annotations_df_drop.head()

1561


,Hospital.Admission.ID,subject.id,chart.time,cohort,Obesity,Non.Adherence,Developmental.Delay.Retardation,Advanced.Heart.Disease,Advanced.Lung.Disease,Schizophrenia.and.other.Psychiatric.Disorders,Alcohol.Abuse,Other.Substance.Abuse,Chronic.Pain.Fibromyalgia,Chronic.Neurological.Dystrophies,Advanced.Cancer,Depression,Dementia,Unsure
0,118003,3644,118003,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0
1,177830,97736,999999,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,185673,27694,999999,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,131938,16275,131938,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,198999,4059,198999,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0


In [31]:
hadmids = list(annotations_df_drop["Hospital.Admission.ID"])
subjids = list(annotations_df_drop["subject.id"])
print(len(hadmids), len(set(hadmids)), len(set(subjids)))

1561 1561 1045


## Join Patients and ICU stays table

In [42]:
pat_icu_df = pd.merge(patients_df, icustays_df, on='SUBJECT_ID')
print(len(pat_icu_df))
pat_icu_df.head()

61532


,SUBJECT_ID,GENDER,DOB,EXPIRE_FLAG,HADM_ID,ICUSTAY_ID,INTIME,LOS
0,249,F,2075-03-13 00:00:00,0,116935,215044,2149-12-18 20:06:02,5.7262
1,249,F,2075-03-13 00:00:00,0,149546,269035,2155-02-03 20:17:29,1.9282
2,249,F,2075-03-13 00:00:00,0,149546,263055,2155-02-07 18:51:16,3.8815
3,249,F,2075-03-13 00:00:00,0,158975,282599,2156-05-01 18:10:12,2.0233
4,249,F,2075-03-13 00:00:00,0,158975,263882,2156-05-10 17:47:35,1.0614


In [46]:
## Verifying if merge worked properly

for i in range(len(icustays_df)):
    if(icustays_df['SUBJECT_ID'][i]==249):
        print(icustays_df['SUBJECT_ID'][i], icustays_df['HADM_ID'][i], icustays_df['ICUSTAY_ID'][i])

249 116935 215044
249 149546 269035
249 149546 263055
249 158975 282599
249 158975 263882


## Keep only the entries with matching Hospital Adm ID

In [51]:
pat_icu_df_anno = pat_icu_df[pat_icu_df['HADM_ID'].isin(hadmids)]
print(len(pat_icu_df_anno), len(list(set(list(pat_icu_df_anno['HADM_ID'])))), 
      len(list(set(list(pat_icu_df_anno['SUBJECT_ID'])))),
      len(list(set(list(pat_icu_df_anno['ICUSTAY_ID'])))))
pat_icu_df_anno.head()

1652 1531 1034 1652


,SUBJECT_ID,GENDER,DOB,EXPIRE_FLAG,HADM_ID,ICUSTAY_ID,INTIME,LOS
64,690,M,2109-09-24 00:00:00,1,166781,238871,2182-04-24 07:05:00,3.4500
65,690,M,2109-09-24 00:00:00,1,152820,241997,2182-09-12 23:31:00,1.9938
66,690,M,2109-09-24 00:00:00,1,135389,262991,2188-02-11 20:12:46,2.9975
67,690,M,2109-09-24 00:00:00,1,135389,217830,2188-02-24 17:05:38,5.9279
84,704,M,2095-12-26 00:00:00,1,151175,227550,2176-03-31 02:44:00,14.6417


## First ICU entry per Hospital Admission

In [67]:
pat_icu_df_anno.loc[:, 'INTIME'] = pd.to_datetime(pat_icu_df_anno['INTIME'])
pat_icu_df_anno_first = pat_icu_df_anno.loc[pat_icu_df_anno.groupby('HADM_ID')['INTIME'].idxmin()].reset_index(drop=True)
print(len(pat_icu_df_anno_first), len(list(set(list(pat_icu_df_anno_first['HADM_ID'])))), 
      len(list(set(list(pat_icu_df_anno_first['SUBJECT_ID'])))),
      len(list(set(list(pat_icu_df_anno_first['ICUSTAY_ID'])))))
pat_icu_df_anno_first.head()

1531 1531 1034 1531


,SUBJECT_ID,GENDER,DOB,EXPIRE_FLAG,HADM_ID,ICUSTAY_ID,INTIME,LOS
0,9973,M,2084-05-02 00:00:00,1,100020,282580,2142-11-30 23:06:12,1.1011
1,3365,F,2107-01-05 00:00:00,0,100103,200434,2179-12-13 11:36:35,3.2836
2,27290,M,2033-12-10 00:00:00,1,100137,212691,2115-01-12 23:14:06,3.7297
3,9882,M,2096-02-11 00:00:00,0,100177,251800,2151-01-31 06:11:00,6.5389
4,5525,M,2050-04-03 00:00:00,1,100473,257484,2115-02-14 01:30:22,5.7583


## Calculating Age

In [75]:
pat_icu_df_anno_first['AGE'] = pd.to_datetime(pat_icu_df_anno_first['INTIME']).dt.year - pd.to_datetime(pat_icu_df_anno_first['DOB']).dt.year
pat_icu_df_anno_first.head()

,SUBJECT_ID,GENDER,DOB,EXPIRE_FLAG,HADM_ID,ICUSTAY_ID,INTIME,LOS,AGE
0,9973,M,2084-05-02 00:00:00,1,100020,282580,2142-11-30 23:06:12,1.1011,58
1,3365,F,2107-01-05 00:00:00,0,100103,200434,2179-12-13 11:36:35,3.2836,72
2,27290,M,2033-12-10 00:00:00,1,100137,212691,2115-01-12 23:14:06,3.7297,82
3,9882,M,2096-02-11 00:00:00,0,100177,251800,2151-01-31 06:11:00,6.5389,55
4,5525,M,2050-04-03 00:00:00,1,100473,257484,2115-02-14 01:30:22,5.7583,65


## Merge with Annotations 

In [76]:
pat_icu_anno_df = pd.merge(pat_icu_df_anno_first, annotations_df_drop, left_on='HADM_ID', right_on='Hospital.Admission.ID', how='inner')
print(len(pat_icu_anno_df))
pat_icu_anno_df.head()

1531


,SUBJECT_ID,GENDER,DOB,EXPIRE_FLAG,HADM_ID,ICUSTAY_ID,INTIME,LOS,AGE,Hospital.Admission.ID,...,Advanced.Lung.Disease,Schizophrenia.and.other.Psychiatric.Disorders,Alcohol.Abuse,Other.Substance.Abuse,Chronic.Pain.Fibromyalgia,Chronic.Neurological.Dystrophies,Advanced.Cancer,Depression,Dementia,Unsure
0,9973,M,2084-05-02 00:00:00,1,100020,282580,2142-11-30 23:06:12,1.1011,58,100020,...,0,0,0,0,1,1,0,0,0,0
1,3365,F,2107-01-05 00:00:00,0,100103,200434,2179-12-13 11:36:35,3.2836,72,100103,...,0,0,0,0,0,0,0,0,0,0
2,27290,M,2033-12-10 00:00:00,1,100137,212691,2115-01-12 23:14:06,3.7297,82,100137,...,0,0,0,0,0,0,0,0,0,0
3,9882,M,2096-02-11 00:00:00,0,100177,251800,2151-01-31 06:11:00,6.5389,55,100177,...,0,0,0,0,0,0,0,0,0,0
4,5525,M,2050-04-03 00:00:00,1,100473,257484,2115-02-14 01:30:22,5.7583,65,100473,...,0,0,0,0,0,0,0,0,0,0


In [77]:
pat_icu_anno_df = pat_icu_anno_df.drop(columns=['DOB', 'INTIME', 'Hospital.Admission.ID', 'subject.id','chart.time']).reset_index(drop=True)
pat_icu_anno_df.head()

,SUBJECT_ID,GENDER,EXPIRE_FLAG,HADM_ID,ICUSTAY_ID,LOS,AGE,cohort,Obesity,Non.Adherence,...,Advanced.Lung.Disease,Schizophrenia.and.other.Psychiatric.Disorders,Alcohol.Abuse,Other.Substance.Abuse,Chronic.Pain.Fibromyalgia,Chronic.Neurological.Dystrophies,Advanced.Cancer,Depression,Dementia,Unsure
0,9973,M,1,100020,282580,1.1011,58,1,0,0,...,0,0,0,0,1,1,0,0,0,0
1,3365,F,0,100103,200434,3.2836,72,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,27290,M,1,100137,212691,3.7297,82,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9882,M,0,100177,251800,6.5389,55,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5525,M,1,100473,257484,5.7583,65,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [78]:
pat_icu_anno_df.to_csv("../PATIENTS_INFORMATION.csv")